In [29]:
################## IMPORT LIBRARIES ##################

from IPython.display import Audio
import numpy as np
import pandas as pd
from os.path import join as pjoin
import scipy.signal as sig
from IPython.display import Audio
from scipy.io import wavfile
import soundfile as sf
import os
import sys


In [2]:
################## IMPORT MY MODULES ##################
sys.path.append('../src')

# import my modules:
import helpers as hlp
import importlib
importlib.reload(hlp);

In [22]:
# parameters: 
N_irs=2000 # number of rirs clones to generate
fixed_mic_dist=0.1
fs_target=48000

# Note: Mic characteristics, bands, and other parameters of the rir generation 
# are defined inside the funcion "render_random_rir" (this could be changed)

# load metadata used for training reverb transfer network
df_metadata=pd.read_csv("~/joanna/reverb-match-cond-u-net/dataset-metadata/17-05-2024--15-42_data_set.csv",index_col=0)
# chose only test split and reset the index of the original metadata df
df_metadata=df_metadata[df_metadata["split"]=="test"].reset_index(drop=True)


In [23]:
# ---------------- GENERATE RIR CLONES WITH MASP BASED ON METADATA ---------------

from multiprocessing import Pool, Manager

def render_clone_ir(df_metadata,i):
    # Multiprocessing function which, based on the room info,
    # computes room simulation, renders and stores a RIR.
    # I use this function to generate an extension of the test dataset:
    # RIRs from the same rooms as original RIRa but at different locations
    # I call them RIR clones.
    # Inputs:
    # df_metadata - input dataframe with dataset metadata (including existing RIRs)
    # i - index for a specific RIR
    # -------------------------------------------------------

    mic_clone_rir=hlp.render_random_rir(df_metadata["room_x"][i],df_metadata["room_y"][i],df_metadata["room_z"][i],df_metadata["rt60_set"][i],fixed_mic_dist=0.1)
    original_rir_path=df_metadata["ir_file_path"][i]
    dir_name = os.path.dirname(original_rir_path)
    file_name = os.path.basename(original_rir_path)
    # Add the prefix to the file name
    clone_file_name = "clone_" + file_name
    # Join the directory name with the new base name to get the new path
    clone_rir_path = os.path.join(dir_name, clone_file_name)
    wavfile.write(clone_rir_path, fs_target, np.squeeze(mic_clone_rir).astype(np.float32))
    print("save file:" + clone_rir_path)

if __name__ == "__main__":
    with Pool(processes=8) as pool:
        # run render_clone_ir in parallel
        pool.starmap(render_clone_ir, [(df_metadata, idx) for idx in range(N_irs)])

Compute echogram: Source 0 - Receiver 0
Apply receiver direcitivites
Apply absorption: Source 0 - Receiver 0
Rendering echogram: Source 0 - Receiver 0
     Filtering and combining bands
save file:/home/ubuntu/Data/synth_rirs_mono/clone_monoRIR_x2993y2674z268_rtms88.wav
Compute echogram: Source 0 - Receiver 0
Compute echogram: Source 0 - Receiver 0

Apply receiver direcitivitesApply absorption: Source 0 - Receiver 0Apply receiver direcitivites

Apply absorption: Source 0 - Receiver 0
Compute echogram: Source 0 - Receiver 0
Rendering echogram: Source 0 - Receiver 0
Rendering echogram: Source 0 - Receiver 0Compute echogram: Source 0 - Receiver 0

Apply receiver direcitivites

Apply receiver direcitivitesApply absorption: Source 0 - Receiver 0
Apply absorption: Source 0 - Receiver 0
Rendering echogram: Source 0 - Receiver 0
Rendering echogram: Source 0 - Receiver 0
Compute echogram: Source 0 - Receiver 0Compute echogram: Source 0 - Receiver 0

Compute echogram: Source 0 - Receiver 0Apply r

In [30]:
# ----- LISTEN TO REVERBERANT AUDIOS: -----
for i in range(3):
    original_rir_path=df_metadata["ir_file_path"][i]
    dir_name = os.path.dirname(original_rir_path)
    file_name = os.path.basename(original_rir_path)
    clone_file_name = "clone_" + file_name
    # original impulse response
    rir_orig, fs_rir = sf.read(pjoin(dir_name,file_name)) 
    # cloned impulse response
    rir_clone, fs_rir = sf.read(pjoin(dir_name,clone_file_name)) 
    # speech
    speech_data, fs_speech = sf.read(df_metadata.loc[10,"speech_file_path"]) 
    # create reverberant version of speech 
    revspeech_orig=sig.fftconvolve(speech_data,rir_orig, 'full', 0)
    revspeech_clone=sig.fftconvolve(speech_data,rir_clone, 'full', 0)
    # print corresponding rt60
    print("t60="+ str(df_metadata.loc[i,"rt60_set"]))
    # audio player
    display(Audio(data=revspeech_orig,rate=fs_speech))
    display(Audio(data=revspeech_clone,rate=fs_speech))

t60=0.8763285556667173


t60=0.206841540378053


t60=0.5130468285373271
